# Task2: SimpleHRNet

### Drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Clone the repo and install the dependencies

In [2]:
# clone the repo
!git clone https://github.com/stefanopini/simple-HRNet.git

Cloning into 'simple-HRNet'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 420 (delta 53), reused 85 (delta 30), pack-reused 302
Receiving objects: 100% (420/420), 78.08 MiB | 27.12 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [3]:
%cd simple-HRNet
!pwd

/content/simple-HRNet
/content/simple-HRNet


In [4]:
# install requirements
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 12.5 MB/s eta 0:00:00


### Download HRNet pre-trained weights

In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
# download weights

# create weights folder
%cd /content/simple-HRNet
!mkdir weights
%cd /content/simple-HRNet/weights

# download weights pose_hrnet_w48_384x288.pth
!gdown 1UoJhTtjHNByZSm96W3yFTfU5upJnsKiS

%cd /content/simple-HRNet

/content/simple-HRNet
/content/simple-HRNet/weights
Downloading...
From: https://drive.google.com/uc?id=1UoJhTtjHNByZSm96W3yFTfU5upJnsKiS
To: /content/simple-HRNet/weights/pose_hrnet_w48_384x288.pth
100% 255M/255M [00:00<00:00, 270MB/s]
/content/simple-HRNet


### Download the result file of Task 1
큰 파일 크기로 분할해 저장했던 task1의 결과를 불러옴.



In [ ]:
!mkdir data

In [ ]:
!unzip '/content/drive/MyDrive/해커톤데이터구축/Cropped_train004.zip' -d '/content/simple-HRNet/data'

Archive:  /content/drive/MyDrive/해커톤데이터구축/Cropped_train004.zip
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0002.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0003.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0012.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0013.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0015.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0018.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0019.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0020.PNG  
  inflating: /content/simple-HRNet/data/Train004/F01_P001_A1_W001_D2022-08-23-10-12-00_0021.PNG  
  inflating: /content/simple-HRNet/data/Train

In [ ]:
import os
imgs = os.listdir('/content/simple-HRNet/data/Train004')
imgs = [file for file in imgs if file.endswith(".PNG")]

In [ ]:
len(imgs)

3678

### Distinguish dulplicated data & Run
Colab으로 실행 시 RAM 소진으로 run 중간에 끊기는 경우가 발생함. <br>
따라서 재시도 시 중복 데이터를 구별해 새로운 데이터만을 HRNet에 적용함.

In [ ]:
# 처리한 데이터 리스트
import os
done = os.listdir('/content/drive/MyDrive/해커톤데이터구축/Task2Result_train')
done = [file for file in done if file.endswith(".PNG")]

In [ ]:
# 처리한 데이터 수
len(done)

11974

In [ ]:
# 아직 처리하지 않은 데이터 리스트
notyet = list(set(imgs) - set(done))

In [ ]:
# 처리 전 데이터 수
len(notyet)

0

In [ ]:
# Test
import cv2
import requests
import matplotlib.pyplot as plt
import torch
from skimage import io
from PIL import Image
from SimpleHRNet import SimpleHRNet

%matplotlib inline
from misc.visualization import joints_dict

def plot_joints(ax, output):
    bones = joints_dict()["coco"]["skeleton"]
    # bones = joints_dict()["mpii"]["skeleton"]

    for bone in bones:
        xS = [output[:,bone[0],1], output[:,bone[1],1]]
        yS = [output[:,bone[0],0], output[:,bone[1],0]]
        ax.plot(xS, yS, linewidth=3, c=(0,0.3,0.7))
    ax.scatter(joints[:,:,1],joints[:,:,0], s=20, c='r')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # singleperson, COCO weights
model = SimpleHRNet(48, 17, "./weights/pose_hrnet_w48_384x288.pth", multiperson=False, device=device)


for i, img in enumerate(notyet):
  image = io.imread('/content/simple-HRNet/data/Train004/'+img)

  joints = model.predict(image)

  fig = plt.figure(figsize=(60/2.54, 30/2.54))
  ax = fig.add_subplot(121)
  ax.imshow(Image.open('/content/simple-HRNet/data/Train004/'+img))
  plot_joints(ax, joints)
  plt.axis('off')
  plt.savefig(fname=('/content/drive/MyDrive/해커톤데이터구축/Task2Result_train/'+img),bbox_inches='tight', pad_inches=0)
  print(f'{i+1}/{len(notyet)}') # check the process

### Option: Save as a zip file

In [ ]:
# zip으로 다시 한 번 저장
# !zip -r /content/drive/MyDrive/해커톤데이터구축/Task2Result_val.zip /content/drive/MyDrive/해커톤데이터구축/Task2Result_val/*

### Remove NaN files
누락된 annotation 정보를 가진 파일 삭제


In [ ]:
nan = ['F01_P001_A2_W001_D2022-08-23-10-23-00_0553.PNG',
'F02_P001_A3_W001_D2022-08-24-11-14-35_1224.PNG',
'F05_P001_A3_W001_D2022-09-23-10-07-45_0861.PNG',
'F05_P001_A1_W001_D2022-09-23-10-31-57_0132.PNG',
'F02_P001_A2_W001_D2022-08-24-11-14-47_1643.PNG',
'F05_P001_A2_W001_D2022-09-23-10-04-45_0143.PNG',
'F05_P001_A1_W001_D2022-09-23-10-31-57_0668.PNG',
'F05_P001_A3_W001_D2022-09-23-10-23-27_0154.PNG',
'F05_P001_A1_W001_D2022-09-23-10-09-01_0751.PNG',
'F05_P001_A3_W001_D2022-09-23-10-07-45_1062.PNG',
'F05_P001_A3_W001_D2022-09-23-10-07-45_1176.PNG',
'F02_P001_A1_W001_D2022-08-24-11-15-55_1280.PNG',
'F05_P001_A3_W001_D2022-09-23-10-07-45_1011.PNG',
'F05_P001_A3_W001_D2022-09-23-10-07-45_1180.PNG',
'F05_P001_A3_W001_D2022-09-23-10-23-27_0392.PNG',
'F05_P001_A1_W001_D2022-09-23-10-31-57_0007.PNG',
'F05_P001_A3_W001_D2022-09-23-10-07-45_0896.PNG',
'F05_P001_A1_W001_D2022-09-23-10-31-57_0887.PNG',
'F02_P001_A3_W001_D2022-08-24-11-14-35_0362.PNG',
'F02_P001_A3_W001_D2022-08-24-11-14-35_0870.PNG']

In [ ]:
!mkdir test

In [ ]:
!unzip '/content/drive/MyDrive/해커톤데이터구축/Task2Result_test.zip' -d '/content/simple-HRNet/test'

Archive:  /content/drive/MyDrive/해커톤데이터구축/Task2Result_test.zip
   creating: /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/
  inflating: /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0004.PNG  
  inflating: /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0006.PNG  
  inflating: /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0011.PNG  
  inflating: /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0032.PNG  
  inflating: /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0039.PNG  
  inflating: /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-

In [ ]:
test = os.listdir('/content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg')
test = [file for file in test]

In [ ]:
val = os.listdir('/content/drive/MyDrive/해커톤데이터구축/Task2Result_val')
val = [file for file in val]

In [ ]:
train = os.listdir('/content/drive/MyDrive/해커톤데이터구축/Task2Result_train')
train = [file for file in train]

test로 분류된 이미지 중 누락값 제거

In [ ]:
nan_test = list(set(test) & set(nan))
nan_test

['F05_P001_A3_W001_D2022-09-23-10-07-45_1062.PNG',
 'F05_P001_A1_W001_D2022-09-23-10-09-01_0751.PNG',
 'F01_P001_A2_W001_D2022-08-23-10-23-00_0553.PNG',
 'F05_P001_A1_W001_D2022-09-23-10-31-57_0668.PNG']

In [ ]:
for img_name in nan_test:
  os.remove('/content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/'+img_name)

In [ ]:
# 삭제 확인
nan_test = list(set(test) & set(nan))
nan_test

[]

In [ ]:
# option: save it to the zip file
!zip -r /content/drive/MyDrive/해커톤데이터구축/Task2Result_test_final.zip /content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/*

  adding: content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0004.PNG (deflated 0%)
  adding: content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0006.PNG (deflated 0%)
  adding: content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0011.PNG (deflated 0%)
  adding: content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0032.PNG (deflated 0%)
  adding: content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0039.PNG (deflated 0%)
  adding: content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_P001_A1_W001_D2022-08-23-10-12-00_0044.PNG (deflated 0%)
  adding: content/simple-HRNet/test/content/drive/MyDrive/worker_data/Task2Result_testimg/F01_

validation으로 분류된 이미지 중 누락값 제거

In [ ]:
nan_val = list(set(val) & set(nan))
nan_val

['F02_P001_A2_W001_D2022-08-24-11-14-47_1643.PNG',
 'F05_P001_A3_W001_D2022-09-23-10-07-45_1180.PNG',
 'F05_P001_A3_W001_D2022-09-23-10-23-27_0392.PNG',
 'F02_P001_A3_W001_D2022-08-24-11-14-35_0870.PNG',
 'F02_P001_A3_W001_D2022-08-24-11-14-35_1224.PNG']

In [ ]:
for img_name in nan_val:
  os.remove('/content/drive/MyDrive/해커톤데이터구축/Task2Result_val/'+img_name)

In [ ]:
# 삭제 확인
nan_val = list(set(val) & set(nan))
nan_val

[]

In [ ]:
# option: save it to the zip file
!zip -r /content/drive/MyDrive/해커톤데이터구축/Task2Result_val_final.zip /content/drive/MyDrive/해커톤데이터구축/Task2Result_val/*

  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00_0001.PNG (deflated 0%)
  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00_0005.PNG (deflated 0%)
  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00_0014.PNG (deflated 0%)
  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00_0016.PNG (deflated 0%)
  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00_0017.PNG (deflated 0%)
  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00_0027.PNG (deflated 0%)
  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00_0031.PNG (deflated 0%)
  adding: content/drive/MyDrive/해커톤데이터구축/Task2Result_val/F01_P001_A1_W001_D2022-08-23-10-12-00

train으로 분류된 이미지 중 누락값 제거

In [ ]:
nan_train = list(set(train) & set(nan))
nan_train

['F05_P001_A3_W001_D2022-09-23-10-07-45_1011.PNG',
 'F05_P001_A3_W001_D2022-09-23-10-07-45_1176.PNG',
 'F05_P001_A1_W001_D2022-09-23-10-31-57_0132.PNG',
 'F02_P001_A3_W001_D2022-08-24-11-14-35_0362.PNG',
 'F02_P001_A1_W001_D2022-08-24-11-15-55_1280.PNG',
 'F05_P001_A1_W001_D2022-09-23-10-31-57_0007.PNG',
 'F05_P001_A3_W001_D2022-09-23-10-07-45_0861.PNG',
 'F05_P001_A3_W001_D2022-09-23-10-07-45_0896.PNG',
 'F05_P001_A1_W001_D2022-09-23-10-31-57_0887.PNG',
 'F05_P001_A2_W001_D2022-09-23-10-04-45_0143.PNG',
 'F05_P001_A3_W001_D2022-09-23-10-23-27_0154.PNG']

In [ ]:
for img_name in nan_train:
  os.remove('/content/drive/MyDrive/해커톤데이터구축/Task2Result_train/'+img_name)

In [ ]:
# 삭제 확인 
nan_train = list(set(train) & set(nan))
nan_train

[]